### IMPORTANT!

First of all it's important to import the necessary libraries to work with SQL in a Jupyter Notebook

In [1]:
import sqlite3
import prettytable 

con = sqlite3.connect('library.db')
cur = con.cursor()

prettytable.DEFAULT = 'DEFAULT'

%load_ext sql
%sql sqlite:///library.db


My database is empty, so next I will create tables and insert some

In [2]:
%%sql

-- Create Tables
CREATE TABLE Books (
    book_id INTEGER PRIMARY KEY, -- Auto-increment in SQLite
    title TEXT NOT NULL,
    isbn TEXT UNIQUE NOT NULL,
    published_date DATE,
    genre TEXT
);

CREATE TABLE Authors (
    author_id INTEGER PRIMARY KEY, -- Auto-increment in SQLite
    name TEXT NOT NULL,
    nationality TEXT,
    birth_date DATE
);

CREATE TABLE BookAuthors (
    book_id INTEGER NOT NULL,
    author_id INTEGER NOT NULL,
    PRIMARY KEY (book_id, author_id),
    FOREIGN KEY (book_id) REFERENCES Books(book_id),
    FOREIGN KEY (author_id) REFERENCES Authors(author_id)
);

CREATE TABLE Members (
    member_id INTEGER PRIMARY KEY, -- Auto-increment in SQLite
    name TEXT NOT NULL,
    email TEXT UNIQUE NOT NULL,
    join_date DATE NOT NULL,
    membership_status TEXT CHECK(membership_status IN ('Active', 'Expired', 'Suspended')) DEFAULT 'Active'
);

CREATE TABLE Loans (
    loan_id INTEGER PRIMARY KEY, -- Auto-increment in SQLite
    book_id INTEGER NOT NULL,
    member_id INTEGER NOT NULL,
    loan_date DATE NOT NULL,
    due_date DATE NOT NULL,
    return_date DATE,
    FOREIGN KEY (book_id) REFERENCES Books(book_id),
    FOREIGN KEY (member_id) REFERENCES Members(member_id)
);

-- Insert Data into Books
INSERT INTO Books (title, isbn, published_date, genre) VALUES
('Machine Learning Basics', '9780451524119', '2022-08-14', 'Technology'),
('The Art of Fiction', '9780671027030', '2019-04-22', 'Literature'),
('Space Exploration', '9781402894626', '2021-06-30', 'Science'),
('Modern Economics', '9780393048253', '2020-02-15', 'Economics'),
('Culinary Arts', '9780060525476', '2023-01-10', 'Cooking'),
('Environmental Science', '9780545227247', '2022-09-05', 'Science');

-- Insert Data into Authors
INSERT INTO Authors (name, nationality, birth_date) VALUES
('James Patterson', 'American', '1965-09-22'),
('Maria Sanchez', 'Spanish', '1988-05-30'),
('David Kim', 'Korean', '1975-11-15'),
('Emily White', 'Canadian', '1992-07-04');

-- Insert Data into Members
INSERT INTO Members (name, email, join_date, membership_status) VALUES
('Grace Taylor', 'grace.t@lib.com', '2023-02-28', 'Active'),
('Henry Clark', 'henry.c@lib.com', '2022-12-15', 'Suspended'),
('Charlotte Lewis', 'charlotte@lib.com', '2023-04-01', 'Active'),
('Ethan Walker', 'ethan.w@lib.com', '2023-03-10', 'Expired'),
('Mia Hall', 'mia.h@lib.com', '2023-05-05', 'Active');

-- Query to Retrieve Auto-Generated IDs
-- Run these queries to get the actual IDs for Books, Authors, and Members.
SELECT book_id, title FROM Books;
SELECT author_id, name FROM Authors;
SELECT member_id, name FROM Members;

-- Insert Data into BookAuthors
-- Replace the IDs below with the actual IDs retrieved from the queries above.
INSERT INTO BookAuthors (book_id, author_id) VALUES
(1, 4), -- Machine Learning Basics by Emily White
(2, 3), -- The Art of Fiction by David Kim
(3, 2), -- Space Exploration by Maria Sanchez
(4, 1), -- Modern Economics by James Patterson
(5, 4), -- Culinary Arts by Emily White
(6, 3); -- Environmental Science by David Kim

-- Insert Data into Loans
-- Replace the IDs below with the actual IDs retrieved from the queries above.
INSERT INTO Loans (book_id, member_id, loan_date, due_date, return_date) VALUES
(3, 3, '2023-04-10', '2023-04-24', '2023-04-23'), -- Space Exploration loaned to Charlotte Lewis
(4, 4, '2023-05-01', '2023-05-15', NULL),         -- Modern Economics loaned to Ethan Walker
(5, 5, '2023-05-10', '2023-05-24', NULL),         -- Culinary Arts loaned to Mia Hall
(6, 1, '2023-05-12', '2023-05-26', '2023-05-25'), -- Environmental Science loaned to Grace Taylor
(1, 2, '2023-05-15', '2023-05-29', NULL),         -- Machine Learning Basics loaned to Henry Clark
(2, 3, '2023-05-18', '2023-06-01', '2023-05-30'), -- The Art of Fiction loaned to Charlotte Lewis
(3, 5, '2023-06-05', '2023-06-19', NULL),         -- Space Exploration loaned to Mia Hall
(4, 4, '2023-06-10', '2023-06-24', NULL);         -- Modern Economics loaned to Ethan Walker

 * sqlite:///library.db
Done.
Done.
Done.
Done.
Done.
6 rows affected.
4 rows affected.
5 rows affected.
Done.
Done.
Done.
6 rows affected.
8 rows affected.


[]

---

NOTE: These tasks are designed to test advanced SQL skills, including joins, aggregations, filtering, and the use of CTEs. Each task requires careful thought and a deep understanding of SQL syntax and logic.

---



### **Task 1: Identify Overdue Books**
**Description:**  
Find all books that are currently overdue (i.e., `return_date` is `NULL` and `due_date` has passed). Include details about the book title, member name, loan date, due date, and the number of days overdue.

**Tables Involved:**  
- `Loans`
- `Books`
- `Members`

**Expected Output:**  
| Book Title              | Member Name       | Loan Date   | Due Date     | Days Overdue |
|-------------------------|-------------------|-------------|--------------|--------------|
| Modern Economics        | Ethan Walker      | 2023-05-01  | 2023-05-15   | 15           |
| Culinary Arts           | Mia Hall          | 2023-05-10  | 2023-05-24   | 14           |

**Hint:** Use `CURRENT_DATE` to calculate the number of days overdue.

---



In [3]:
%%sql

WITH Dates AS (
    SELECT
        loan_id,
        book_id,
        member_id,
        loan_date,
        due_date,
        return_date,
        CURRENT_DATE AS today_date,
        CASE
            WHEN return_date IS NULL AND due_date < CURRENT_DATE THEN julianday(CURRENT_DATE) - julianday(due_date)
            ELSE 0
        END AS days_overdue
    FROM
        Loans
) 
SELECT
    b.title AS book_title,
    m.name AS member_name,
    d.loan_date,
    d.due_date,
    d.days_overdue
FROM
    Dates d
    JOIN Members m ON d.member_id = m.member_id
    JOIN Books b ON d.book_id = b.book_id
WHERE
    days_overdue > 0



 * sqlite:///library.db
Done.


book_title,member_name,loan_date,due_date,days_overdue
Modern Economics,Ethan Walker,2023-05-01,2023-05-15,669.0
Culinary Arts,Mia Hall,2023-05-10,2023-05-24,660.0
Machine Learning Basics,Henry Clark,2023-05-15,2023-05-29,655.0
Space Exploration,Mia Hall,2023-06-05,2023-06-19,634.0
Modern Economics,Ethan Walker,2023-06-10,2023-06-24,629.0


### **Task 2: Most Prolific Author**
**Description:**  
Identify the author who has written the most books. Include the author's name, nationality, and the count of books they have authored.

**Tables Involved:**  
- `Authors`
- `BookAuthors`

**Expected Output:**  
| Author Name    | Nationality | Book Count |
|----------------|-------------|------------|
| David Kim      | Korean      | 2          |

**Hint:** Use aggregation (`COUNT`) and sorting (`ORDER BY`) to find the top author.

---



In [4]:
%%sql

WITH BookCount AS (
    SELECT
        a.name,
        a.nationality,
        COUNT(DISTINCT ba.book_id) AS book_count
    FROM
        Authors a
        JOIN BookAuthors ba ON a.author_id = ba.author_id
    GROUP BY
        a.name
)

SELECT
    name AS "Author Name",
    nationality AS Nationality,
    book_count AS "Book Count" 
FROM
    BookCount
WHERE
    book_count = (
        SELECT
            MAX(book_count)
        FROM
            BookCount
    )

 * sqlite:///library.db
Done.


Author Name,Nationality,Book Count
David Kim,Korean,2
Emily White,Canadian,2


### **Task 3: Members with Multiple Loans**
**Description:**  
Find members who have borrowed more than one book. Include the member's name, email, and the count of loans.

**Tables Involved:**  
- `Members`
- `Loans`

**Expected Output:**  
| Member Name       | Email               | Loan Count |
|-------------------|---------------------|------------|
| Charlotte Lewis   | charlotte@lib.com   | 2          |
| Ethan Walker      | ethan.w@lib.com     | 2          |

**Hint:** Use `GROUP BY` and `HAVING` to filter members with multiple loans.

---



In [5]:
%%sql

SELECT
    m.name AS "Member Name",
    m.email AS Email,
    COUNT(DISTINCT l.loan_id) AS "Loan Count"
FROM
    Members m
    JOIN Loans l ON m.member_id = l.member_id
GROUP BY
    m.name,
    m.email
HAVING
    COUNT(DISTINCT l.loan_id) > 1

 * sqlite:///library.db
Done.


Member Name,Email,Loan Count
Charlotte Lewis,charlotte@lib.com,2
Ethan Walker,ethan.w@lib.com,2
Mia Hall,mia.h@lib.com,2


### **Task 4: Genre Popularity**
**Description:**  
Determine the popularity of each genre based on the number of loans. Include the genre name and the total number of loans for that genre.

**Tables Involved:**  
- `Books`
- `Loans`

**Expected Output:**  
| Genre       | Total Loans |
|-------------|-------------|
| Science     | 3           |
| Technology  | 1           |

**Hint:** Join `Books` and `Loans`, then aggregate by genre.

---



In [6]:
%%sql

SELECT
    b.genre AS Genre,
    COUNT(l.loan_id) AS "Total Loans"
FROM
    Books b
    JOIN Loans l ON b.book_id = l.book_id
GROUP BY
    b.genre
ORDER BY
    COUNT(l.loan_id) DESC


 * sqlite:///library.db
Done.


Genre,Total Loans
Science,3
Economics,2
Technology,1
Literature,1
Cooking,1


### **Task 5: Active Members with No Loans**
**Description:**  
Find active members who have not borrowed any books. Include their name, email, and join date.

**Tables Involved:**  
- `Members`
- `Loans`

**Expected Output:**  
| Member Name       | Email               | Join Date   |
|-------------------|---------------------|-------------|
| Grace Taylor      | grace.t@lib.com     | 2023-02-28  |

**Hint:** Use a `LEFT JOIN` between `Members` and `Loans`, and filter for `loan_id IS NULL`.

---



In [7]:
%%sql

SELECT
    m.name AS "Member Name",
    m.email AS Email,
    m.join_date AS "Join Date"
FROM
    Members m
LEFT JOIN
    Loans l ON m.member_id = l.member_id
WHERE
    l.loan_id IS NULL
    AND m.membership_status = 'Active';

 * sqlite:///library.db
Done.


Member Name,Email,Join Date


### **Task 6: Average Loan Duration by Genre**
**Description:**  
Calculate the average loan duration (in days) for each genre. Only include genres with at least two loans.

**Tables Involved:**  
- `Books`
- `Loans`

**Expected Output:**  
| Genre       | Avg Loan Duration (Days) |
|-------------|---------------------------|
| Science     | 12                        |

**Hint:** Use a CTE to calculate loan durations, then aggregate by genre.

---



In [8]:
%%sql


WITH LoanDuration AS (
    SELECT
        l.loan_id,
        CASE
            WHEN l.return_date IS NOT NULL THEN julianday(l.return_date) - julianday(l.loan_date)
            ELSE julianday('now') - julianday(l.loan_date)
        END AS loan_duration,
        b.genre
    FROM 
        Loans l
        JOIN Books b ON l.book_id = b.book_id
)
SELECT
    genre AS Genre,
    AVG(loan_duration) AS "Avg Loan Duration (Days)"
FROM
    LoanDuration
GROUP BY
    genre
HAVING
    COUNT(loan_id) >= 2


 * sqlite:///library.db
Done.


Genre,Avg Loan Duration (Days)
Economics,663.5138004166074
Science,224.83793347220248


### **Task 7: Books Never Loaned**
**Description:**  
List all books that have never been loaned. Include the book title, ISBN, and genre.

**Tables Involved:**  
- `Books`
- `Loans`

**Expected Output:**  
| Book Title              | ISBN             | Genre       |
|-------------------------|------------------|-------------|
| The Art of Fiction      | 9780671027030    | Literature  |

**Hint:** Use a `LEFT JOIN` between `Books` and `Loans`, and filter for `loan_id IS NULL`.

---



In [9]:
%%sql

SELECT
    b.title AS "Book Title",
    isbn AS ISBN,
    genre AS Genre
FROM
    Books b
    LEFT JOIN Loans l ON b.book_id = l.book_id
WHERE
    l.loan_id IS NULL



 * sqlite:///library.db
Done.


Book Title,ISBN,Genre


### **Task 8: Monthly Loan Trends**
**Description:**  
Generate a report showing the number of loans per month for the year 2023. Include the month name and the loan count.

**Tables Involved:**  
- `Loans`

**Expected Output:**  
| Month       | Loan Count |
|-------------|------------|
| April       | 2          |
| May         | 4          |
| June        | 2          |

**Hint:** Use `strftime('%m', loan_date)` to extract the month and aggregate by it.

---



In [10]:
%%sql

SELECT
    strftime('%m', loan_date) AS Month,
    COUNT(loan_id) AS "Loan Count"
FROM
    Loans
WHERE
    strftime('%Y', loan_date) = '2023'
GROUP BY
    Month
ORDER BY
    Month



 * sqlite:///library.db
Done.


Month,Loan Count
04,1
05,5
06,2


### **Task 9: Suspended Members with Outstanding Loans**
**Description:**  
Find suspended members who still have outstanding loans (i.e., `return_date IS NULL`). Include the member's name, email, and the titles of the books they have not returned.

**Tables Involved:**  
- `Members`
- `Loans`
- `Books`

**Expected Output:**  
| Member Name       | Email               | Book Title          |
|-------------------|---------------------|---------------------|
| Henry Clark       | henry.c@lib.com     | Machine Learning Basics |

**Hint:** Use a `JOIN` between `Members`, `Loans`, and `Books`, and filter by `membership_status = 'Suspended'`.

---



In [11]:
%%sql

SELECT
    m.name,
    m.email,
    b.title
FROM
    Members m 
    JOIN Loans l ON m.member_id = l.member_id
    JOIN Books b ON l.book_id = b.book_id
WHERE
    l.return_date IS NULL
    AND m.membership_status = 'Suspended'

 * sqlite:///library.db
Done.


name,email,title
Henry Clark,henry.c@lib.com,Machine Learning Basics


### **Task 10: Author Contribution to Genres**
**Description:**  
For each author, determine the percentage of books they have contributed to each genre. Include the author's name, genre, and the contribution percentage.

**Tables Involved:**  
- `Authors`
- `BookAuthors`
- `Books`

**Expected Output:**  
| Author Name    | Genre       | Contribution (%) |
|----------------|-------------|------------------|
| David Kim      | Science     | 50.0             |
| Emily White    | Technology  | 100.0            |

**Hint:** Use a CTE to calculate total books per genre, then calculate the contribution percentage.

---

In [12]:
%%sql

WITH TotalBooksPerGenre AS (
    SELECT
        genre,
        COUNT(book_id) AS total_books
    FROM
        Books
    GROUP BY
        genre
), TotalBooksPerAuthor AS (
    SELECT
        a.name,
        b.genre,
        COUNT(b.book_id) AS count_books_per_author
    FROM
        Books b
        JOIN BookAuthors ba ON b.book_id = ba.book_id
        JOIN Authors a ON ba.author_id = a.author_id
    GROUP BY
        a.name,
        b.genre
)
SELECT
    ba.name AS "Author Name",
    ba.genre AS Genre,
    (CAST(ba.count_books_per_author AS REAL) / bg.total_books) * 100 AS "Contribution (%)"
FROM
    TotalBooksPerAuthor ba
    JOIN TotalBooksPerGenre bg ON ba.genre = bg.genre

 * sqlite:///library.db
Done.


Author Name,Genre,Contribution (%)
David Kim,Literature,100.0
David Kim,Science,50.0
Emily White,Cooking,100.0
Emily White,Technology,100.0
James Patterson,Economics,100.0
Maria Sanchez,Science,50.0


---

### About the Author

**Name:** Sebastian Mondragon  

- **Email:** basmondragon@proton.me
- **Telegram:** [https://t.me/basmondragon](https://t.me/basmondragon)
- **LinkedIn:** [https://www.linkedin.com/in/basmondragon/](https://www.linkedin.com/in/basmondragon/)

#### Skills

- **Programming Languages:** Python, SQL  
- **Libraries & Frameworks:** Pandas, NumPy, Scikit-learn, Matplotlib, Seaborn, XGBoost  
- **Methodologies:** Data Cleaning, Feature Engineering, Machine Learning, Model Evaluation  
- **Soft Skills:** Problem-Solving, Analytical Thinking, Communication

#### Next Steps

If you have any feedback or suggestions for improving this project, feel free to reach out to me via email or LinkedIn. I’m always open to learning and collaborating on new ideas!  

Feel free to explore my other projects on GitHub: [https://github.com/basmondragon](https://github.com/basmondragon)